In [1]:
import pandas as pd
import datetime as dt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.api import VARMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import warnings

# Define the start and end dates
start = dt.datetime(2021, 6, 1)
end = dt.datetime(2022, 1, 5)
symbol = 'AXISBANK.NS'

# Fetch stock data
stk_data = yf.download(symbol, start=start, end=end)
stk_data = stk_data[['Open', 'High', 'Low', 'Close']]

[*********************100%%**********************]  1 of 1 completed


In [2]:
import numpy as np
np.random.seed(42)
exog_data = pd.DataFrame({
    'Exog1': np.random.randn(len(stk_data)),
    'Exog2': np.random.randn(len(stk_data))
}, index=stk_data.index)

In [3]:
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(stk_data)
data_normalized = pd.DataFrame(data_normalized, columns=["Open", "High", "Low", "Close"])

In [4]:
exog_normalized = MinMaxScaler().fit_transform(exog_data)
exog_normalized = pd.DataFrame(exog_normalized, columns=["Exog1", "Exog2"])

In [5]:
test_obs = 29
train_data = data_normalized[:-test_obs]
test_data = data_normalized[-test_obs:]
train_exog = exog_normalized[:-test_obs]
test_exog = exog_normalized[-test_obs:]


In [ ]:
aic_vals = []
bic_vals = []
orders = range(1, 6)
for p in orders:
    model = VARMAX(train_data, exog=train_exog, order=(p, 0))
    result = model.fit(method='nm', maxiter=1000, disp=True)  # 'nm' stands for Nelder-Mead
    aic_vals.append(result.aic)
    bic_vals.append(result.bic)
    print(f'Order = {p}, AIC = {result.aic}, BIC = {result.bic}')


Order = 1, AIC = -1997.585094281275, BIC = -1891.3450535485988
Order = 2, AIC = -2003.2624104187557, BIC = -1852.2897209565317
Order = 3, AIC = -1970.9605672285838, BIC = -1775.255229036812
Order = 4, AIC = -1939.2496478988287, BIC = -1698.8116609775088


In [ ]:
# Select the best order based on AIC
best_order = orders[aic_vals.index(min(aic_vals))]
print(f'Best Order by AIC: {best_order}')


In [ ]:
# Fit the selected VARMAX model
model = VARMAX(train_data, exog=train_exog, order=(best_order, 0))
result = model.fit(disp=False)
